In [1]:
import boto3
import json
import os

In [2]:
with open("credentials.json", 'r', encoding="utf8") as cred:
    json_cred = json.load(cred)

In [3]:
aws_secret_access_key = json_cred["aws_secret_access_key"]
aws_access_key_id = json_cred["aws_access_key_id"]
region = json_cred["region"]

###### Initialize AWS S3 Remote Connection

In [2]:
s3_client = boto3.client('s3')

###### Create AWS S3 Bucket

In [3]:
bucket_name = "pytalk-demo"
response = s3_client.create_bucket(ACL='private', 
                    Bucket=bucket_name,
                    CreateBucketConfiguration={'LocationConstraint': 'ap-south-1'})

In [4]:
path = "C:\\Users\\javed\\aws\\aws_files"
files = [file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))]

###### Uplaod Local Folder Files to AWS S3 Bucket

In [5]:
for file in files:
    filepath = os.path.join(path, file)
    print(filepath)
    print(f'Upload file: {file} to S3 Bucket: {bucket_name}')
    response = s3_client.upload_file(filepath, bucket_name, file)

print(f'All files uploaded to S3 Bucket.')

C:\Users\javed\aws\aws_files\1_page.pdf
Upload file: 1_page.pdf to S3 Bucket: pytalk-demo
C:\Users\javed\aws\aws_files\2_page.pdf
Upload file: 2_page.pdf to S3 Bucket: pytalk-demo
All files uploaded to S3 Bucket.


###### Get the AWS S3 Bucket Files Details

In [8]:
# get the list of files that has been uploaded to S3 bucket 
response = s3_client.list_objects_v2(
    Bucket= bucket_name,
)
response

{'ResponseMetadata': {'RequestId': 'FA2WPHFMATBWCFWC',
  'HostId': 'y25WxVH2way6ivbyjj1DjybLYf1AkcUZAdL3+VzeiiFz1btn59dQRv05ji9VLfHdygffGV6AaFc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'y25WxVH2way6ivbyjj1DjybLYf1AkcUZAdL3+VzeiiFz1btn59dQRv05ji9VLfHdygffGV6AaFc=',
   'x-amz-request-id': 'FA2WPHFMATBWCFWC',
   'date': 'Mon, 15 Nov 2021 16:24:33 GMT',
   'x-amz-bucket-region': 'ap-south-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': '1_page.pdf',
   'LastModified': datetime.datetime(2021, 11, 15, 16, 19, 36, tzinfo=tzutc()),
   'ETag': '"a700486237b9aca6fc51cd3ee7f47d05"',
   'Size': 231737,
   'StorageClass': 'STANDARD'},
  {'Key': '2_page.pdf',
   'LastModified': datetime.datetime(2021, 11, 15, 16, 19, 36, tzinfo=tzutc()),
   'ETag': '"a3810b25ece1dd19ea43b5675a838206"',
   'Size': 253048,
   'StorageClass': 'STANDARD'}],
 'Name': 'pytalk-demo',

###### Delete AWS S3 Files and Bucket 

In [23]:
# delete the s3 bucket files one-by-one 
while response['KeyCount'] > 0:
    print('Deleting %d objects from bucket %s' % (len(response['Contents']),bucket_name))
    response = s3_client.delete_objects(
        Bucket=bucket_name,
        Delete={
            'Objects':[{'Key':obj['Key']} for obj in response['Contents']]
        }
    )
    response = s3_client.list_objects_v2(
        Bucket=bucket_name,
    )

print(f'All files has been deleted. Bucekt {bucket_name} is empty.')

Deleting 2 objects from bucket pytalk-demo
All files has been deleted. Bucekt pytalk-demo is empty.


In [24]:
# Delete the Bucket once it is empty
print(f'Now deleting Bucket {bucket_name}')
response = s3_client.delete_bucket(
    Bucket=bucket_name
)

Now deleting Bucket pytalk-demo
